In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/colab_env/lib/python3.10/site-packages')
!pip install portalocker>=2.0.0

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
#Importing the libraires

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
import torch.optim as optim
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import numericalize_tokens_from_iterator
from torchinfo import summary
from torchtext.data.functional import to_map_style_dataset,simple_space_split


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import portalocker
import regex as re

In [ ]:
tokenizer = get_tokenizer(tokenizer='basic_english')
tokenizer

<function torchtext.data.utils._basic_english_normalize(line)>

In [ ]:
from datasets import load_dataset
dataset = load_dataset('wikitext','wikitext-2-raw-v1',split='train')

In [ ]:
def clean_text(text):
  text = str(text).lower()
  text = re.sub('[\{\}\.,:;)(=\'\"@]','',text)
  text = re.sub('\n','',text)
  text = re.sub('戦場のヴァルキュリア3','',text)
  text = re.sub('  ',' ',text)
  return str(text)

cleaned_dataset = [clean_text(text) for text in dataset['text']]

In [ ]:
#Building Vocab from iterator and dataset

vocab = build_vocab_from_iterator(map(tokenizer,cleaned_dataset),specials=['<UNK>','<PAD>'],min_freq = 20)
len(vocab)

8214

In [ ]:
#looking at the first few tokesn in the vocab
vocab.set_default_index(vocab["<UNK>"])
vocab.get_itos()[:10]

['<UNK>', '<PAD>', 'the', 'of', 'and', 'in', 'to', 'a', 'was', '-']

In [ ]:
token_iterator = simple_space_split(cleaned_dataset)
count=0
for i in token_iterator:
  print(i)
  count=count+1
  if count==4:
    break

[]
['valkyria', 'chronicles', 'iii']
[]
['senjō', 'no', 'valkyria', '3', 'unrecorded', 'chronicles', 'japanese', 'lit', 'valkyria', 'of', 'the', 'battlefield', '3', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'is', 'a', 'tactical', 'role', '-', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'mediavision', 'for', 'the', 'playstation', 'portable', 'released', 'in', 'january', '2011', 'in', 'japan', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '-', 'time', 'gameplay', 'as', 'its', 'predecessors', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', 'nameless', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are', 'pitted', 'against', 'the', 'imperial', 'unit', 'cala

In [ ]:
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_iterator)

In [ ]:
len(cleaned_dataset)

36718

In [ ]:
sequence_list = [[num for num in id]for id in sequence]
len(sequence_list)

36714

In [ ]:
def generate_cbow_data_context(sequence_list, context_size):
    for sequence in sequence_list:
        for j in range(context_size, len(sequence) - context_size):
            context = sequence[j - context_size: j] + sequence[j + 1: j + context_size + 1]
            yield context

context_generator = generate_cbow_data_context(sequence_list, 2)

def generate_cbow_data_target(sequence_list, context_size):
    for sequence in sequence_list:
        for j in range(context_size, len(sequence) - context_size):
            target = sequence[j]
            yield target
target_generator = generate_cbow_data_target(sequence_list,2)


In [ ]:
context = next(iter(context_generator))
target = next(iter(target_generator))

In [ ]:
context,target

([2, 61, 362, 5], 130)

In [ ]:
context_tensors = torch.tensor(list(context_generator))
target_tensors = torch.tensor(list(target_generator))

In [ ]:
#Making Tensor Dataset
train_dataset = TensorDataset(context_tensors, target_tensors)

In [ ]:
train_dataloader = DataLoader(train_dataset,shuffle=False,batch_size=500)

In [ ]:
feature,target = next(iter(train_dataloader))
feature.shape,target.shape

(torch.Size([500, 4]), torch.Size([500]))

In [ ]:
#Model Building

class WordEmbeddings(nn.Module):
  def __init__(self):
    super().__init__()

    self.embeddings = nn.Embedding(num_embeddings = len(vocab),
                                   embedding_dim = 300)
    self.linear_1 = nn.Linear(in_features = 300,
                            out_features = len(vocab))

  def forward(self,x):
    x = self.embeddings(x)
    x = x.mean(axis=1)
    x = self.linear_1(x)
    return x

In [ ]:
#Model Instantiation

model = WordEmbeddings()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.95)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
summary(model)

Layer (type:depth-idx)                   Param #
WordEmbeddings                           --
├─Embedding: 1-1                         2,465,100
├─Linear: 1-2                            2,473,317
Total params: 4,938,417
Trainable params: 4,938,417
Non-trainable params: 0

In [ ]:
#device agnostic code

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
#Training loop
from tqdm.auto import tqdm

training_loss = []
for epoch in tqdm(range(11)):
  train_loss = 0
  model.train()
  for feature,label in train_dataloader:
    model = model.to(device)
    feature = feature.to(device)
    label = label.to(device)

    y_train_pred = model(feature)

    loss = loss_fn(y_train_pred,label)
    train_loss = train_loss + loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
  train_loss = train_loss/len(train_dataloader)
  training_loss.append(train_loss)
  print(f"Epoch:{epoch} | Training Loss : {train_loss}")

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
embedding_matrix = model.embeddings.state_dict()['weight']

In [ ]:
vocab.get_stoi()['man']

In [ ]:
vocab.get_stoi()['king']

In [ ]:
vocab.get_stoi()['woman']

In [ ]:
vocab.get_stoi()['black']

In [ ]:
man_matrix = embedding_matrix[238].cpu().detach().numpy()
king_matrix = embedding_matrix[286].cpu().detach().numpy()
woman_matrix = embedding_matrix[862].cpu().detach().numpy()
queen_matrix = embedding_matrix[1095].cpu().detach().numpy()
fruit_matrix = embedding_matrix[1558].cpu().detach().numpy()
orange_matrix = embedding_matrix[4999].cpu().detach().numpy()
green_matrix = embedding_matrix[1209].cpu().detach().numpy()
red_matrix = embedding_matrix[425].cpu().detach().numpy()
black_matrix = embedding_matrix[285].cpu().detach().numpy()

In [ ]:
from sklearn.decomposition import PCA

model_ml = PCA(n_components = 2)

In [ ]:
df = pd.DataFrame([man_matrix,king_matrix,woman_matrix,queen_matrix,fruit_matrix,orange_matrix,green_matrix,red_matrix,black_matrix])

In [ ]:
df.head()

In [ ]:
df_reduced = model_ml.fit_transform(df)
df_reduced = pd.DataFrame(df_reduced)

df_reduced['label'] = ['man','king','woman','queen','fruit','orange','green','red','black']
df_reduced.head()

In [ ]:
sns.scatterplot(data=df_reduced,x=0,y=1)
for i in range(len(df_reduced)):
    plt.annotate(df_reduced['label'][i], (df_reduced.iloc[i, 0], df_reduced.iloc[i, 1]))
plt.show()